## Setup & Dependencies
This cell imports all necessary libraries and modules. We include pandas for data manipulation, sklearn components (like OneHotEncoder, RobustScaler, ColumnTransformer, Pipeline, and SimpleImputer) for building the preprocessing pipeline, and crucial custom functions from our local functions package.

We also modify sys.path to ensure the notebook can successfully find the custom modules within the project structure.

In [11]:
#Imports

import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath('..'))
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from functions.clean_df import (load_and_combine_csvs,
                                            clean_dataframe,
                                            add_confidential_flags)

from functions.state_imput import apply_state_estimation
from functions.feature_engineering import feature_engineering
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

## Data Ingestion
This step uses the custom function load_and_combine_csvs to read and consolidate all individual monthly CPGF CSV files located in the raw_data directory. The resulting df_raw DataFrame serves as the initial, unprocessed dataset for the entire pipeline. It also automatically adds the ARQUIVO_ORIGEM column for traceability.

In [12]:
csv_path = '../raw_data'
df_raw = load_and_combine_csvs(csv_path)

## Data Cleaning and Initial Feature Engineering
This cell sequentially applies the foundational cleaning and transformation functions:

clean_dataframe: Converts the 'VALOR TRANSAÇÃO' column to numeric (handling the Brazilian decimal format) and converts 'DATA TRANSAÇÃO' to datetime format. It also removes duplicate rows.

add_confidential_flags: Creates one binary (0/1) feature: 'SIGILOSO' (marks confidential transactions).

apply_state_estimation: Executes the custom logic to infer the ESTADO_ESTIMADO (Estimated State) of the transaction based on text patterns in the agency names, which is critical for geospatial risk analysis.

In [13]:
df_limpo = clean_dataframe(df_raw)
df_conf = add_confidential_flags(df_limpo)
df_final = apply_state_estimation(df_conf)

## Advanced Feature Engineering

This section creates derived and contextual features crucial for training the anomaly detection models. These transformations aim to normalize the target variable and capture temporal patterns, periodicity, and contextual financial behavior (i.e., is this transaction normal for this specific entity at this specific time?).

1. Atomic and Structural Features

These features involve simple transformations of existing columns for identification or basic temporal context:

ID_PORTADOR: Creates a unique identifier by concatenating the CPF and NOME PORTADOR. This ensures that even if two people share the same name or two entries share the same CPF, the combination acts as a unique categorical key for the transaction holder.

FIM_SEMANA (Weekend Flag): Creates a binary flag (1/0) indicating if the transaction occurred on a weekend (day 5 or 6). Crucially, it is set to 0 for all confidential (sigiloso) transactions to avoid creating a false pattern based on hidden data.

LOG_VALOR: Applies the $\log(1+x)$ transformation to VALOR TRANSAÇÃO. This is essential to reduce the extreme positive skew in financial data, stabilizing the distribution for better model training.

In [ ]:
df_final = feature_engineering(df_final)

'\n### ID Portador\ndf_final["ID_PORTADOR"] = df_final["CPF PORTADOR"] + df_final["NOME PORTADOR"]\n\n### Weekend\ndf_final["FIM_SEMANA"] = np.where(df_final[\'SIGILOSO\'] == 1, 0, df_final[\'DATA TRANSAÇÃO\'].dt.day_of_year.isin([5, 6]).astype(int))\n\n## Log valor da transação\ndf_final[\'LOG_VALOR\'] = np.log1p(df_final[\'VALOR TRANSAÇÃO\'])\n'

2. Frequency and Periodicity Features
These features capture the intrinsic popularity and temporal grouping of various entities:

FREQ_NOME ÓRGÃO, FREQ_ESTADO_ESTIMADO, FREQ_NOME FAVORECIDO: Calculates the relative frequency of occurrence for key categorical fields. This gives the model a numerical measure of how common or rare an entity is (e.g., a rarely seen NOME FAVORECIDO might be inherently riskier).

SEMESTRE_EXTRATO: Groups transactions into the first or second semester (1 or 2). This captures broader annual seasonality that is often missed when only using monthly data.

3. Contextual Ratio Features (Anomaly Signal)
These are the most valuable features for anomaly detection, as they compare an individual transaction against the average behavior of its peer group in the same time period.

MEDIA_VALOR_ORGAO_MES / MEDIA_VALOR_ORGAO_SEM: Calculates the average transaction value grouped by NOME ÓRGÃO for both the month and the semester. This establishes the "normal" financial baseline for that agency during that specific time period.

RATIO_MES / RATIO_SEM: Calculates the ratio of the individual VALOR TRANSAÇÃO to its corresponding group average (monthly/semesterly).

Interpretation: A ratio near 1.0 indicates a transaction close to the historical norm. A ratio significantly above 1.0 signals an outlier transaction (e.g., a transaction value 5 times the group average), creating a strong, normalized anomaly signal for the model.

In [17]:
## Trasnformar em função ##

## Category frequencies
cols_categoria = ['NOME ÓRGÃO', 'ESTADO_ESTIMADO', 'NOME FAVORECIDO']

for col in cols_categoria:
    freq_map = df_final[col].value_counts(normalize=True)
    df_final[f'FREQ_{col}'] = df_final[col].map(freq_map)

# Average órgão per month
df_final['MEDIA_VALOR_ORGAO_MES'] = df_final.groupby(['NOME ÓRGÃO', 'ANO EXTRATO', 'MÊS EXTRATO'])['VALOR TRANSAÇÃO'].transform('mean')

# Ratio valor da transação/ average órgão in the period
df_final['RATIO_MES'] = df_final['VALOR TRANSAÇÃO'] / df_final['MEDIA_VALOR_ORGAO_MES']

## Validation Checks
These commands are executed after the cleaning and initial feature engineering to perform essential data validation.

df_final.info(): Displays the non-null counts and data types for the final DataFrame, confirming that the VALOR TRANSAÇÃO column is now a numeric type (float64) and checking the fill rate of the newly engineered columns like ESTADO_ESTIMADO.

df_final.head(): Shows the first few rows of the processed data, allowing a quick visual inspection of the results from the clean_dataframe and apply_state_estimation functions.

In [18]:
# Example checks
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307307 entries, 0 to 392087
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   CÓDIGO ÓRGÃO SUPERIOR   307307 non-null  int64         
 1   NOME ÓRGÃO SUPERIOR     307307 non-null  object        
 2   CÓDIGO ÓRGÃO            307307 non-null  int64         
 3   NOME ÓRGÃO              307307 non-null  object        
 4   CÓDIGO UNIDADE GESTORA  307307 non-null  int64         
 5   NOME UNIDADE GESTORA    307307 non-null  object        
 6   ANO EXTRATO             307307 non-null  int64         
 7   MÊS EXTRATO             307307 non-null  int64         
 8   CPF PORTADOR            289417 non-null  object        
 9   NOME PORTADOR           307307 non-null  object        
 10  CNPJ OU CPF FAVORECIDO  307307 non-null  int64         
 11  NOME FAVORECIDO         307307 non-null  object        
 12  TRANSAÇÃO               307307 non-

In [19]:
# Example checks
df_final.columns

Index(['CÓDIGO ÓRGÃO SUPERIOR', 'NOME ÓRGÃO SUPERIOR', 'CÓDIGO ÓRGÃO',
       'NOME ÓRGÃO', 'CÓDIGO UNIDADE GESTORA', 'NOME UNIDADE GESTORA',
       'ANO EXTRATO', 'MÊS EXTRATO', 'CPF PORTADOR', 'NOME PORTADOR',
       'CNPJ OU CPF FAVORECIDO', 'NOME FAVORECIDO', 'TRANSAÇÃO',
       'DATA TRANSAÇÃO', 'VALOR TRANSAÇÃO', 'ARQUIVO_ORIGEM', 'SIGILOSO',
       'ESTADO_ESTIMADO', 'ID_portador', 'fim_semana', 'LOG_VALOR',
       'FREQ_NOME ÓRGÃO', 'FREQ_ESTADO_ESTIMADO', 'FREQ_NOME FAVORECIDO',
       'MEDIA_VALOR_ORGAO_MES', 'RATIO_MES'],
      dtype='object')

In [20]:
# Example checks
df_final.head()

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,ANO EXTRATO,MÊS EXTRATO,CPF PORTADOR,NOME PORTADOR,...,SIGILOSO,ESTADO_ESTIMADO,ID_portador,fim_semana,LOG_VALOR,FREQ_NOME ÓRGÃO,FREQ_ESTADO_ESTIMADO,FREQ_NOME FAVORECIDO,MEDIA_VALOR_ORGAO_MES,RATIO_MES
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2025,1,***.725.752-**,VIVIANE CORREA LIMA,...,0,UNIÃO,***.725.752-**VIVIANE CORREA LIMA,0,3.938470,0.006521,0.216305,0.000046,473.572361,0.106298
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2025,1,***.384.652-**,CARLOS EPAMINONDAS GOMES DA SILVA,...,0,UNIÃO,***.384.652-**CARLOS EPAMINONDAS GOMES DA SILVA,0,6.356108,0.006521,0.216305,0.000091,473.572361,1.214176
2,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2025,1,***.562.861-**,ANTONIO CARLOS MELO DOS SANTOS,...,0,UNIÃO,***.562.861-**ANTONIO CARLOS MELO DOS SANTOS,0,5.219815,0.006521,0.216305,0.000013,473.572361,0.388325
3,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2025,1,***.945.361-**,CLAUDELI CONCEICAO DOS SANTOS,...,0,UNIÃO,***.945.361-**CLAUDELI CONCEICAO DOS SANTOS,0,5.327876,0.006521,0.216305,0.016303,473.572361,0.432880
4,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2025,1,***.945.361-**,CLAUDELI CONCEICAO DOS SANTOS,...,0,UNIÃO,***.945.361-**CLAUDELI CONCEICAO DOS SANTOS,0,7.467942,0.006521,0.216305,0.000003,473.572361,3.695317


## Feature Categorization

This cell organizes the DataFrame columns into three logical lists for use in the subsequent Scikit-learn preprocessing pipeline (ColumnTransformer). These lists define the specialized transformations applied to each feature group:

Categorical Columns (cat_columns): These are identifier and nominal features (e.g., NOME ÓRGÃO, CNPJ OU CPF FAVORECIDO). CNPJ OU CPF FAVORECIDO is included here because, despite being stored as an integer, it represents a unique supplier identifier and must be encoded, not scaled. These features will be imputed (to handle missing categories like CPF PORTADOR) and One-Hot Encoded.

Numerical Value (num_value): This contains the single, highly sensitive column, VALOR TRANSAÇÃO.

Numerical Columns (num_columns): This contains discrete codes and time variables (e.g., CÓDIGO ÓRGÃO, MÊS EXTRATO).

Transformation Rationale (Why Separate VALOR TRANSAÇÃO?)
We separate VALOR TRANSAÇÃO for a critical two-step process due to its extreme positive skew (many small transactions, few large ones):
Log Transformation ($\log(1+x)$): Applied first, this compresses the huge range of values, reducing the skew and normalizing the distribution. This is vital for improving the performance of distance-based models like the Autoencoder and LOF.
Robust Scaling (RobustScaler): This is applied to all numerical features (num_value and num_columns). The RobustScaler uses the median and Interquartile Range (IQR), making the scaling highly resilient to outliers. 

This is essential for Project Jacurutu, as the anomalies we are trying to detect are the outliers themselves.

Note: We exclude the datetime column (DATA TRANSAÇÃO) as temporal information is covered by the existing discrete columns (ANO EXTRATO and MÊS EXTRATO).

In [ ]:
# Identifying columns
cat_columns = [
    'NOME ÓRGÃO SUPERIOR', 'NOME ÓRGÃO', 'NOME UNIDADE GESTORA', 'CPF PORTADOR',
    'NOME PORTADOR', 'NOME FAVORECIDO', 'TRANSAÇÃO', 'ARQUIVO_ORIGEM', 'ESTADO_ESTIMADO',
    'ID_PORTADOR', 'CNPJ OU CPF FAVORECIDO'
]
# Numerical columns
# The highly sensitive, continuous financial value
num_value = ['LOG_VALOR']

# Discrete codes and less-skewed numerical features
num_columns = [
    'CÓDIGO ÓRGÃO SUPERIOR', 'CÓDIGO ÓRGÃO', 'CÓDIGO UNIDADE GESTORA', 'ANO EXTRATO',
    'MÊS EXTRATO', 'SIGILOSO', 'FIM_DE_SEMANA', 'FREQ_NOME ÓRGÃO',
    'FREQ_ESTADO_ESTIMADO', 'FREQ_NOME FAVORECIDO', 'SEMESTRE_EXTRATO',
    'MEDIA_VALOR_ORGAO_MES', 'MEDIA_VALOR_ORGAO_SEM', 'RATIO_MES', 'RATIO_SEM'
]

## Final Preprocessing Pipeline: preprocessor
The preprocessor object, built using Scikit-learn's ColumnTransformer, orchestrates the final necessary transformations for all features before they are input into the anomaly detection models (e.g., Isolation Forest, LOF and Autoencoder). The pipeline is optimized to handle the unique challenges of financial data: extreme skew, high cardinality, and outlier sensitivity.

1. Categorical Pipeline (cat_pipeline)
This pipeline handles all nominal and identifier columns (cat_columns), including CNPJ OU CPF FAVORECIDO.

Imputation (SimpleImputer): Missing values (like in CPF PORTADOR) are imputed using a constant value ('Desconhecido'). This is the correct approach for categorical data, as it preserves missingness as its own unique category without distorting the data distribution.

Encoding (OneHotEncoder): Transforms nominal variables (strings) into a format usable by models (binary vectors). The setting handle_unknown='ignore' prevents the pipeline from failing if a new, unseen category appears in the test or prediction set.

2. Specialized Value Pipeline (num_value_pipeline)
This pipeline is exclusively applied to the highly sensitive feature, VALOR TRANSAÇÃO.

Log Transformation (FunctionTransformer(np.log1p)): This is the crucial step for mitigating skew. Applying the natural logarithm of (1 + x) compresses the extremely wide range of financial values, bringing the distribution closer to normal. This improves the performance and stability of distance-based models.

Robust Scaling (RobustScaler): Scales the log-transformed data. This scaler is preferred over StandardScaler because it uses the median and Interquartile Range (IQR), making it highly resistant to outliers. Since anomalies are the outliers we are looking for, preserving their position relative to the majority of the data is essential.

3. General Numerical Pipeline (num__columns_pipeline)
This pipeline handles the remaining numerical and discrete features (num_columns), such as code numbers, ANO EXTRATO, and flag variables (SIGILOSO).

Robust Scaling (RobustScaler): Since these columns are assumed to have no missing values, they go directly to the scaler. The RobustScaler is used here again to ensure that even subtle outliers in these secondary features do not excessively influence the feature space.

4. ColumnTransformer Integration
The ColumnTransformer applies each specialized pipeline to its designated column subset (num_columns, num_value, cat_columns).

remainder='passthrough': This critical setting ensures that any columns not explicitly listed in the feature lists (e.g., specific transaction IDs, which are vital for merging results later) are kept intact and appended to the final processed array.

In [ ]:
# Numerical columns pipeline: only scaling (assuming no NaNs)
num__columns_pipeline = RobustScaler()

# Numerical value pipeline: Log-transform and scale (assuming no NaNs)
num_value_pipeline = RobustScaler()

# Categorical pipeline: impute and encode
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Desconhecido')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine in a column transformer
preprocessor = ColumnTransformer([
    ('num_columns', num__columns_pipeline, num_columns),
    ('num-value', num_value_pipeline, num_value),
    ('cat', cat_pipeline, cat_columns),
],
remainder='passthrough'
)

### **Final Update**
#### **Handling Categorical Features**
We excluded raw categorical features (e.g., NOME FAVORECIDO, CPF PORTADOR) from the final training pipeline.

Reasoning: These features possess extremely high cardinality (thousands of unique values). Standard techniques like One-Hot Encoding would create a sparse matrix with thousands of dimensions.

#### **Scaler and Categorical Features**
Based on the descriptive statistics of the LOG_VALOR feature, we identified significant outliers even after the logarithmic transformation. The maximum value (12.24) is well above the statistical upper bound (9.01).

Consequently, we opted for RobustScaler. Unlike MinMaxScaler, which is sensitive to outliers and would squash the core data distribution, RobustScaler scales features using statistics that are robust to outliers (Median and IQR). This ensures that our Anomaly Detection models (LOF/Isolation Forest) receive high-quality input data without being distorted by extreme financial values.